<span style="font-size: 27px;"><b>Load the Stock Price Data</b></span>


In [5]:
import pandas as pd
import talib
import pynance as pn
import numpy as np
file_paths = [
    'C:/Users/User/Desktop/10/data/AAPL_historical_data.csv',
    'C:/Users/User/Desktop/10/data/AMZN_historical_data.csv',
    'C:/Users/User/Desktop/10/data/GOOG_historical_data.csv',
    'C:/Users/User/Desktop/10/data/META_historical_data.csv',
    'C:/Users/User/Desktop/10/data/MSFT_historical_data.csv',
    'C:/Users/User/Desktop/10/data/NVDA_historical_data.csv',
    'C:/Users/User/Desktop/10/data/TSLA_historical_data.csv'
]

data_frames = [pd.read_csv(file) for file in file_paths]

for df in data_frames:
    print(df.head())
    print(df.columns)


         Date      Open      High       Low     Close  Adj Close     Volume  \
0  1980-12-12  0.128348  0.128906  0.128348  0.128348   0.098943  469033600   
1  1980-12-15  0.122210  0.122210  0.121652  0.121652   0.093781  175884800   
2  1980-12-16  0.113281  0.113281  0.112723  0.112723   0.086898  105728000   
3  1980-12-17  0.115513  0.116071  0.115513  0.115513   0.089049   86441600   
4  1980-12-18  0.118862  0.119420  0.118862  0.118862   0.091630   73449600   

   Dividends  Stock Splits  
0        0.0           0.0  
1        0.0           0.0  
2        0.0           0.0  
3        0.0           0.0  
4        0.0           0.0  
Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume',
       'Dividends', 'Stock Splits'],
      dtype='object')
         Date      Open      High       Low     Close  Adj Close      Volume  \
0  1997-05-15  0.121875  0.125000  0.096354  0.097917   0.097917  1443120000   
1  1997-05-16  0.098438  0.098958  0.085417  0.086458   0.0864

In [2]:
for df in data_frames:
    print("Columns:", df.columns)
    
    df.rename(columns={
        'open_price': 'Open', 
        'high_price': 'High',
        'low_price': 'Low',
        'closing_price': 'Close',
        'trade_volume': 'Volume'
    }, inplace=True)


Columns: Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume',
       'Dividends', 'Stock Splits'],
      dtype='object')
Columns: Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume',
       'Dividends', 'Stock Splits'],
      dtype='object')
Columns: Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume',
       'Dividends', 'Stock Splits'],
      dtype='object')
Columns: Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume',
       'Dividends', 'Stock Splits'],
      dtype='object')
Columns: Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume',
       'Dividends', 'Stock Splits'],
      dtype='object')
Columns: Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume',
       'Dividends', 'Stock Splits'],
      dtype='object')
Columns: Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume',
       'Dividends', 'Stock Splits'],
      dtype='object')


In [3]:
for df in data_frames:
    df.fillna(method='ffill', inplace=True)  


<span style="font-size: 27px;"><b>Technical Analysis Indicators</b></span>

In [4]:
for i, df in enumerate(data_frames):
    if not all(col in df.columns for col in ['Open', 'High', 'Low', 'Close', 'Volume']):
        print(f"DataFrame {i+1} is missing one or more required columns. Skipping...")
        continue

    df['SMA_50'] = talib.SMA(df['Close'], timeperiod=50)

    df['RSI_14'] = talib.RSI(df['Close'], timeperiod=14)

    df['MACD'], df['MACD_Signal'], df['MACD_Hist'] = talib.MACD(
        df['Close'], 
        fastperiod=12, 
        slowperiod=26, 
        signalperiod=9
    )

    print(f"DataFrame {i+1} with Indicators:")
    print(df[['Date', 'Close', 'SMA_50', 'RSI_14', 'MACD', 'MACD_Signal', 'MACD_Hist']].head(), "\n")

DataFrame 1 with Indicators:
         Date     Close  SMA_50  RSI_14  MACD  MACD_Signal  MACD_Hist
0  1980-12-12  0.128348     NaN     NaN   NaN          NaN        NaN
1  1980-12-15  0.121652     NaN     NaN   NaN          NaN        NaN
2  1980-12-16  0.112723     NaN     NaN   NaN          NaN        NaN
3  1980-12-17  0.115513     NaN     NaN   NaN          NaN        NaN
4  1980-12-18  0.118862     NaN     NaN   NaN          NaN        NaN 

DataFrame 2 with Indicators:
         Date     Close  SMA_50  RSI_14  MACD  MACD_Signal  MACD_Hist
0  1997-05-15  0.097917     NaN     NaN   NaN          NaN        NaN
1  1997-05-16  0.086458     NaN     NaN   NaN          NaN        NaN
2  1997-05-19  0.085417     NaN     NaN   NaN          NaN        NaN
3  1997-05-20  0.081771     NaN     NaN   NaN          NaN        NaN
4  1997-05-21  0.071354     NaN     NaN   NaN          NaN        NaN 

DataFrame 3 with Indicators:
         Date     Close  SMA_50  RSI_14  MACD  MACD_Signal  MACD_Hist

<span style="font-size: 27px;"><b>PyNance for Financial Metrics</b></span>


In [6]:
for i, df in enumerate(data_frames):
   
    if 'Close' not in df.columns:
        print(f"DataFrame {i+1} is missing 'Close' column. Skipping...")
        continue

   
    df['Daily_Return'] = df['Close'].pct_change()

    annualized_return = np.prod(1 + df['Daily_Return'].dropna()) ** (252/len(df)) - 1
    print(f"DataFrame {i+1} Annualized Return: {annualized_return:.2%}")

    volatility = df['Daily_Return'].std() * np.sqrt(252)
    print(f"DataFrame {i+1} Volatility: {volatility:.2%}")

    risk_free_rate = 0.01  
    sharpe_ratio = (annualized_return - risk_free_rate) / volatility
    print(f"DataFrame {i+1} Sharpe Ratio: {sharpe_ratio:.2f}\n")

DataFrame 1 Annualized Return: 18.59%
DataFrame 1 Volatility: 44.28%
DataFrame 1 Sharpe Ratio: 0.40

DataFrame 2 Annualized Return: 31.92%
DataFrame 2 Volatility: 56.24%
DataFrame 2 Sharpe Ratio: 0.55

DataFrame 3 Annualized Return: 23.66%
DataFrame 3 Volatility: 30.67%
DataFrame 3 Sharpe Ratio: 0.74

DataFrame 4 Annualized Return: 27.50%
DataFrame 4 Volatility: 38.54%
DataFrame 4 Sharpe Ratio: 0.69

DataFrame 5 Annualized Return: 24.39%
DataFrame 5 Volatility: 33.54%
DataFrame 5 Sharpe Ratio: 0.70

DataFrame 6 Annualized Return: 36.00%
DataFrame 6 Volatility: 60.11%
DataFrame 6 Sharpe Ratio: 0.58

DataFrame 7 Annualized Return: 42.07%
DataFrame 7 Volatility: 56.94%
DataFrame 7 Sharpe Ratio: 0.72

